### Imports

In [1]:
from primesieve import nth_prime

### RSA Algorithm

1. Take two distinct, large primes p and q (Ideally these have a similar byte-length)
2. Multiply p and q and store the result in n
3. Find the totient for n using the formula φ(n)=(p−1)(q−1)
4. Take an e coprime that is greater, than 1 and less than n
5. Find d using the formula d⋅e≡1modφ(n)

At this point, the pair (e, n) is the public key and the private key (d, n) is the private key.

In [2]:
p = nth_prime(10)
q = nth_prime(15)

In [3]:
n = p*q

In [4]:
print (p,q,n)

29 47 1363


In [5]:
totient = (p-1)*(q-1)

In [6]:
print (totient)

1288


#### Totient

In number theory, Euler's totient function counts the positive integers up to a given integer n that are relatively prime to n. It is written using the Greek letter phi as φ(n) or ϕ(n), and may also be called Euler's phi function. It can be defined more formally as the number of integers k in the range 1 ≤ k ≤ n for which the greatest common divisor gcd(n, k) is equal to 1. The integers k of this form are sometimes referred to as totatives of n.

![title](phi.png)

The line on the top represents distribution of prime numbers. The phi of a prime number is simply the (n-1)
- Phi function is multiplicative (for relatively prime numbers). 
- Therefore, phi of A times B where A and B are prime is (A-1) times (B-1)

#### Coprime

In number theory, two integers a and b are said to be relatively prime, mutually prime, or coprime (also written co-prime) if the only positive integer (factor) that divides both of them is 1. Consequently, any prime number that divides one does not divide the other. This is equivalent to their greatest common divisor (gcd) being 1

In [7]:
from math import gcd
import random

In [8]:
def modinv(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

In [9]:
def coprimes(a):
    l = []
    for x in range(2, a):
        if gcd(a, x) == 1 and modinv(x,a) != None:
            l.append(x)
    for x in l:
        if x == modinv(x,a):
            l.remove(x)
    return l

In [10]:
coprime_list = coprimes(totient)

In [11]:
secure_random = random.SystemRandom()
e = secure_random.choice(coprime_list)

In [12]:
d = modinv(e, totient)

In [13]:
d

403

### Private and Public key pairs

In [14]:
print ('Public key pair:', e, n)

Public key pair: 163 1363


In [15]:
print ('Private key pair:', d, n)

Private key pair: 403 1363


### Test

In [16]:
test = 2**e % n

In [17]:
test

474

In [18]:
test**d % n

2

### Encryption and Decryption

In [19]:
def encrypt(msg, pub, pri, mod):
    chars = [ord(x) for x in list(msg)]
    cipher = []
    for char in chars:
        cipher.append(chr(char**pub%mod))
    return ''.join(cipher)

In [20]:
def decrypt(msg, pub, pri, mod):
    chars = [ord(x) for x in list(msg)]
    cipher = []
    for char in chars:
        cipher.append(chr(char**pri%mod))
    return ''.join(cipher)

In [21]:
cipher = encrypt(msg = 'hello', pub=e, pri=d, mod=n)

In [22]:
decrypt(cipher, pub=e, pri=d, mod=n)

'hello'

### Python library